In [1]:
import csv
import numpy as np
import sys
import os
from osgeo import gdal
current_dir = os.getcwd()
sys.path.append(os.path.join(current_dir,'..','src'))
from util import *

sys.path.append(os.path.join(current_dir,'..','params'))
from load_params import *



In [2]:
#obtention du tableau RVB à partir des 3 images satellites B2,B3,B4 
chemin = os.path.abspath("../image")
dossier = os.path.join(chemin,"Izmir_Turkey")
img = "2017-08-31-00:00_2017-08-31-23:59_Sentinel-2_L1C"

#image 5
#dossier = os.path.join(chemin,"Carrizo_Plain(California,USA)")
#img = "2017-03-28-00:00_2017-03-28-23:59_Sentinel-2_L1C"

#image 4
#dossier = os.path.join(chemin,"Amposta(Spain)2")
#img = "2017-08-04-00:00_2017-08-04-23:59_Sentinel-2_L2A"

#image 3
#dossier = os.path.join(chemin,"Venice(Italy)")
#img = "2017-07-03-00:00_2017-07-03-23:59_Sentinel-2_L2A"

#image2
#dossier = os.path.join(chemin,"Izmir(Turkey)")
#img = "2017-08-31-00:00_2017-08-31-23:59_Sentinel-2_L1C"

#image 1
#dossier = os.path.join(chemin,"Mount_Bromo")
#img = "2016-08-09-00:00_2016-08-09-23:59_Sentinel-2_L1C"

def RVB(root_srcpath, path_name, prod_name):
    bnames = ['B04','B03','B02']
    
    # Obtenir le chemin en entier
    srcpath = os.path.join(root_srcpath,path_name)
    srcfile = os.path.join(srcpath,prod_name+'_'+bnames[0]+'_Raw.tiff')
    src = gdal.Open(srcfile)
    
    # Obtenir les dimensions de l'image
    nx = src.RasterXSize
    ny = src.RasterYSize
    src = None
    
    # Préparer la matrice de sortie
    out_matrix = np.empty((nx * ny, 3), dtype=float)
    
    # Charger les bandes et remplir la matrice
    for i, bname in enumerate(bnames):
        srcfile = os.path.join(srcpath, prod_name + '_' + bname + '_Raw.tiff')
        src = gdal.Open(srcfile)
                
        # Lire la bande entière 
        arr = src.GetRasterBand(1).ReadAsArray().astype(float) 
        src = None
        
        out_matrix[:, i] = arr.flatten()
        
    return out_matrix, nx, ny
RVB, nx, ny = RVB(chemin, dossier, img)

RVB = np.clip(RVB, 0,1)
#print(np.shape(RVB))
print(np.max(RVB),np.min(RVB))

1.0 0.010400000028312206


In [3]:
#convertion de RVB en XYZ
T_8_new = matrix_s2l2ab4b3b2_to_xyzd65_sovdat2019_eq8_new
T_8 = matrix_s2l2ab4b3b2_to_xyzd65_sovdat2019_eq8
T_9 = matrix_s2l2ab4b3b2_to_xyzd65_sovdat2019_eq9

def RVB_to_XYZ(T, A):
    B = A @ T
    return B

XYZ = RVB_to_XYZ(T_9, RVB)
#print(np.shape(XYZ))
print(np.max(XYZ),np.min(XYZ))

1.0879999999999999 0.025353800322860477


In [4]:
# XYZ to sRVB

#standart Martix define by CIE
M = np.array([[3.240625,-1.537208,-0.498629],
             [-0.968931,1.875756,0.041518],
             [0.055710, -0.204021,1.056996]])

sRVB = M@XYZ.T
#print(sRVB)
print(np.max(sRVB),np.min(sRVB))
sRVB = np.clip(sRVB, 0, 1)
#Vérification de la linéarité

# Calculer le rang de la matrice
rang = np.linalg.matrix_rank(sRVB)
print(f"Rang de la matrice : {rang}")

1.000443134 0.010377444900457833
Rang de la matrice : 3


In [5]:
#sRVB en sRVB 8bit
R = sRVB[0]
V = sRVB[1]
B = sRVB[2]

def phi(R):
    for k in range(len(R)):
        if R[k] <= 0.0031308:
            R[k] = R[k]*255*12.92
        else:
            R[k] = 255*(1.055*R[k]**(1/2.4) - 0.055)
    return R

R_8bit = phi(R)
V_8bit = phi(V)
B_8bit = phi(B)
sRVB_8bit = np.array([R_8bit,V_8bit,B_8bit]).T
sRVB_8bit = np.clip(sRVB_8bit, 0, 255).astype(np.uint8)
#print(sRVB_8bit)
print(np.max(sRVB))
#print(np.shape(sRVB_8bit))
        

254.99999999999997


In [6]:
#transformation en tiff
def matrix_to_image(out_matrix, nx, ny, output_filename):
    # Reshape the matrix to its original 2D form
    b4 = out_matrix[:, 0].reshape((ny, nx))  # B04
    b3 = out_matrix[:, 1].reshape((ny, nx))  # B03
    b2 = out_matrix[:, 2].reshape((ny, nx))  # B02

    # Stack the three bands into an RGB image
    rgb_image = np.dstack((b4, b3, b2))

    # Save the image (using GDAL)
    driver = gdal.GetDriverByName('GTiff')
    out_ds = driver.Create(output_filename, nx, ny, 3, gdal.GDT_Byte)
    out_ds.GetRasterBand(1).WriteArray(b4)
    out_ds.GetRasterBand(2).WriteArray(b3)
    out_ds.GetRasterBand(3).WriteArray(b2)
    
    out_ds.FlushCache()
    out_ds = None  # Close file
    

output_filename = "../image/Izmir_Turkey/Izmir_LC1_T9.tiff"
matrix_to_image(sRVB_8bit, nx, ny, output_filename)

In [7]:
#RVB to sRVB
sRVB_direct = np.clip(RVB*255, 0, 255)

#output_filename = "../image/Izmir_Turkey/Izmir_LC1_direct.tiff"
#matrix_to_image(sRVB_direct, nx, ny, output_filename)